**Classification : All Feature Excluding EmoInt**

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
# !git clone https://github.com/SEERNET/EmoInt.git
%cd EmoInt
%cp -r /content/drive/My\ Drive/MCA\ Project/EmoInt/tweetokenize tweetokenize
!python setup.py install
from EmoInt.emoint.featurizers.emoint_featurizer import EmoIntFeaturizer
from EmoInt.tweetokenize.tweetokenize.tokenizer import Tokenizer
%cd /content/drive/My Drive/MCA Project/
!pip install --upgrade gensim
!pip install libsvm

import pandas as pd
from gensim.models import FastText
import re
import numpy as np
from libsvm import svmutil
import libsvm
from sklearn.metrics import confusion_matrix, roc_curve, auc
from scipy.stats import kendalltau

In [0]:
model_summary = FastText.load('project/models-4.1/model_summary.bin')
# ner_tokens = data_ner["ner_tokens"].astype(str).tolist()
arrays = np.load("Final/part_d_features.npy")
combine_f = pd.read_csv('Combined_Features.csv')
len(combine_f)

In [0]:
# !unzip EmoInt.zip

#### SVM for these emotions
import re
category = set(combine_f["Category"].tolist())
post_types = set(combine_f["Post_Type"].tolist())
ner = combine_f["Ner_Token"].astype(str).tolist()
# ner = [re.sub("\"'\]\[(),","", n) for n in ner] 
# print(ner)
headlines= []
for sent_str in combine_f["Headline"]:
#     sent_str = re.sub(r"'", "", str(sent_str).lower())
    tokens = re.sub(r"[^a-z0-9]+", " ", str(sent_str).lower()).split()
    if(tokens == []):
        tokens = "nan"
    headlines.append(tokens)
    
cat_vec = {}
for c in category:
    cat_vec[c] = model_summary.wv[c] 

for c in post_types:
    cat_vec[c] = model_summary.wv[c]
    
head_vec = []
n = len(headlines)
for h in range(n):
    sum1 = np.array([0.0 for i in range(100)])
    for tok in ner[h]:
        sum1+=model_summary.wv[tok[0]]
#     print(sum1)
    sum1 = sum1/len(headlines[h])
#     print(sum1)
    sum1 = np.concatenate((sum1,cat_vec[combine_f["Category"][h]],cat_vec[combine_f["Post_Type"][h]], arrays[h]))
    head_vec.append(sum1)

In [0]:
head_vec = np.array(head_vec)
feat_len = int(0.8*len(head_vec))
train = head_vec[:feat_len]
res_train = combine_f["class"][:feat_len].tolist()
test = head_vec[feat_len:]
res_test = combine_f["class"][feat_len:].tolist()
print(len(test), len(res_test))

In [0]:
model_svm = svmutil.svm_train(res_train, train,'-t 2 -c 2')

In [0]:
y = svmutil.svm_predict(res_test, test, model_svm)

In [0]:
confusion_matrix(res_test, y[0], labels=[0,1])


In [0]:
tn, fp, fn, tp = confusion_matrix(res_test, y[0], labels=[0,1]).ravel()

In [0]:
recall = tp/(tp+fn)
recall

In [0]:
precision = tp/(tp+fp)
precision

In [0]:
f1 = precision*recall*2/(precision+recall)
f1

In [0]:
fpr, tpr, thresholds = roc_curve(res_test, y[0])
AUC = auc(fpr, tpr)
AUC

In [0]:
svmutil.svm_save_model("model_all4_classification.bin",model_svm)
svmutil.svm_train(res_train+res_test, list(train)+list(test),'-v 10 -t 2 -c 2')

**Classification: All Feature Including EmoInt**

In [0]:
featurizer = EmoIntFeaturizer()
tokenizer = Tokenizer()

In [0]:
def get_vector(h):
    
    headlines = []
    i= 0
    for sent_str in h:

        print(i)

        i+=1

        features = featurizer.featurize(str(sent_str), tokenizer)
        
        headlines.append(features)
    
    return headlines
head_vec2 = get_vector(headlines)

In [0]:
hv = []

for i, h in enumerate(list(head_vec)):
  # print(head_vec2[i])
  hv.append(list(h)+ head_vec2[i])
hv = np.array(hv)
hv.shape

In [0]:
feat_len = int(0.8*len(head_vec))
train = hv[:feat_len]
res_train = combine_f["class"][:feat_len].tolist()
test = hv[feat_len:]
res_test = combine_f["class"][feat_len:].tolist()
print(len(test), len(res_test))

In [0]:
model_svm = svmutil.svm_train(res_train, train,'-t 2 -c 2')
y = svmutil.svm_predict(res_test, test, model_svm)

In [0]:
confusion_matrix(res_test, y[0], labels=[0,1])


In [0]:
tn, fp, fn, tp = confusion_matrix(res_test, y[0], labels=[0,1]).ravel()

In [0]:
recall = tp/(tp+fn)
recall

In [0]:
precision = tp/(tp+fp)
precision

In [0]:
f1 = precision*recall*2/(precision+recall)
f1

In [0]:
fpr, tpr, thresholds = roc_curve(res_test, y[0])
AUC = auc(fpr, tpr)
AUC

In [0]:
svmutil.svm_save_model("model_all5_classification.bin",model_svm)
svmutil.svm_train(res_train+res_test, list(train)+list(test),'-v 10 -t 2 -c 2')

**Regression : All Feature Exlcuding EmoInt**

In [0]:
!pip install --upgrade gensim
!pip install libsvm

import pandas as pd
from gensim.models import FastText
import re
import numpy as np
from libsvm import svmutil
import libsvm
from sklearn.metrics import r2_score, mean_absolute_error
from scipy.stats import kendalltau

In [0]:
model_summary = FastText.load('project/models-4.1/model_summary.bin')

In [0]:
data_ner = pd.read_csv("project/data_ner.csv", sep=";")
data_pos = pd.read_csv("project/data_pos.csv")
data_ner.columns

In [0]:
# ner_tokens = data_ner["ner_tokens"].astype(str).tolist()
arrays = np.load("Final/part_d_features.npy")

In [0]:
combine_f = pd.read_csv('Combined_Features.csv')
len(combine_f)

In [0]:
#### SVM for these emotions
import re
category = set(combine_f["Category"].tolist())
post_types = set(combine_f["Post_Type"].tolist())
ner = combine_f["Ner_Token"].astype(str).tolist()
# ner = [re.sub("\"'\]\[(),","", n) for n in ner] 
# print(ner)
headlines= []
for sent_str in combine_f["Headline"]:
#     sent_str = re.sub(r"'", "", str(sent_str).lower())
    tokens = re.sub(r"[^a-z0-9]+", " ", str(sent_str).lower()).split()
    if(tokens == []):
        tokens = "nan"
    headlines.append(tokens)
    
cat_vec = {}
for c in category:
    cat_vec[c] = model_summary.wv[c] 

for c in post_types:
    cat_vec[c] = model_summary.wv[c]
    
head_vec = []
n = len(headlines)
for h in range(n):
    sum1 = np.array([0.0 for i in range(100)])
    for tok in ner[h]:
        sum1+=model_summary.wv[tok[0]]
#     print(sum1)
    sum1 = sum1/len(headlines[h])
#     print(sum1)
    sum1 = np.concatenate((sum1,cat_vec[combine_f["Category"][h]],cat_vec[combine_f["Post_Type"][h]], arrays[h]))
    head_vec.append(sum1)

In [0]:
head_vec = np.array(head_vec)

In [0]:
feat_len = int(0.8*len(head_vec))
train = head_vec[:feat_len]
res_train = list(np.log2(np.array(combine_f["Like Count"][:feat_len].tolist())+1))
test = head_vec[feat_len:]
res_test = list(np.log2(np.array(combine_f["Like Count"][feat_len:].tolist())+1))
print(len(train), len(res_train))

In [0]:
model_svm = svmutil.svm_train(res_train, train,'-s 4 -t 2 -c 2')

In [0]:
y_full = svmutil.svm_predict(res_test+res_train, list(test)+list(train), model_svm)

In [0]:
r2 = r2_score(res_test+res_train,y_full[0])
r2

In [0]:
mae = mean_absolute_error(res_test+res_train, y_full[0])
mae

In [0]:
kt = kendalltau(res_test+res_train, y_full[0])
kt

In [0]:
y_tr = svmutil.svm_predict(res_train, train, model_svm)

In [0]:
r2 = r2_score(res_train,y_tr[0])
r2

In [0]:
mae = mean_absolute_error(res_train, y_tr[0])
mae

In [0]:
kt = kendalltau(res_train, y_tr[0])
kt

In [0]:
svmutil.svm_save_model("model_all4_regression.bin",model_svm)
model_svm_reg = svmutil.svm_train(res_train+res_test, list(train)+list(test),'-s 4 -v 10 -t 2 -c 2')

**Regression : All Feature Inclusding EmoInt**

In [0]:
featurizer = EmoIntFeaturizer()
tokenizer = Tokenizer()

In [0]:
def get_vector(h):
    
    headlines = []
    i= 0
    for sent_str in h:

        # print(i)

        i+=1

        features = featurizer.featurize(str(sent_str), tokenizer)
        
        headlines.append(features)
    
    return headlines
head_vec2 = get_vector(headlines)

In [0]:
hv = []

for i, h in enumerate(list(head_vec)):
  # print(head_vec2[i])
  hv.append(list(h)+ head_vec2[i])
hv = np.array(hv)
hv.shape

In [0]:
feat_len = int(0.8*len(head_vec))
train = hv[:feat_len]
res_train = list(np.log2(np.array(combine_f["Like Count"][:feat_len].tolist())+1))
test = hv[feat_len:]
res_test = list(np.log2(np.array(combine_f["Like Count"][feat_len:].tolist())+1))
print(len(train), len(res_train))

In [0]:
model_svm = svmutil.svm_train(res_train, train,'-s 4 -t 2 -c 2')
y_full = svmutil.svm_predict(res_test+res_train, list(test)+list(train), model_svm)

In [0]:
r2 = r2_score(res_test+res_train,y_full[0])
r2

In [0]:
mae = mean_absolute_error(res_test+res_train, y_full[0])
mae

In [0]:
kt = kendalltau(res_test+res_train, y_full[0])
kt

In [0]:
y_tr = svmutil.svm_predict(res_train, train, model_svm)

In [0]:
r2 = r2_score(res_train,y_tr[0])
r2

In [0]:
mae = mean_absolute_error(res_train, y_tr[0])
mae

In [0]:
kt = kendalltau(res_train, y_tr[0])
kt

In [0]:
svmutil.svm_save_model("model_all5_regression.bin",model_svm)


**Regression : Relevance News Category Features**

In [0]:
!pip install libsvm
import pandas as pd
from gensim.models import FastText
import re
import numpy as np
from libsvm import svmutil
import libsvm

In [0]:
cd /content/drive/My Drive/MCA Project/

In [0]:
data = pd.read_excel("project/NewsHeadlines_01012019_30092019.xlsx")
# ls = [1 if i < 4761 else 0 for i in range(47611)]
# index = data["Like Count"].tolist()
data["Category"].replace({"Tech": "Technology","-":"Uncategorized", "Sports":"Sport"}, inplace=True)
data.head()

In [0]:
train_df = data["Summary"]
train_df = train_df.values.tolist()
summary = []
for sent_str in train_df:
#     sent_str = re.sub(r"'", "", str(sent_str).lower())
    tokens = re.sub(r"[^a-z0-9]+", " ", str(sent_str).lower()).split()
    summary.append(tokens)
model_summary = FastText(summary , size=100, window=5, min_count=5, workers=4,sg=1)
model_summary.wv.most_similar("indian")    

In [0]:
model_summary = FastText.load("project/models-4.1/model_summary.bin")

In [0]:
category = set(data["Category"].tolist())

In [0]:
headlines= []
for sent_str in data["Headline"]:
#     sent_str = re.sub(r"'", "", str(sent_str).lower())
    tokens = re.sub(r"[^a-z0-9]+", " ", str(sent_str).lower()).split()
    if(tokens == []):
        tokens = "nan"
    headlines.append(tokens)

cat_vec = {}
for c in category:
    cat_vec[c] = model_summary[c]

head_vec = []
n = len(headlines)
for h in range(n):
    sum1 = np.array([0.0 for i in range(100)])
    for tok in headlines[h]:
        sum1+=model_summary[tok]
#     print(sum1)
    sum1 = sum1/len(headlines[h])
#     print(sum1)
    sum1 = np.concatenate((sum1,cat_vec[data["Category"][h]]))
    head_vec.append(sum1)

train = head_vec[:int(0.8*(47611))]
res_train = list(np.log2(np.array(data["Like Count"][:int(0.8*47611)].tolist())+1))
test = head_vec[int(0.8*47611):]
res_test = list(np.log2(np.array(data["Like Count"][int(0.8*47611):].tolist())+1))
print(len(train), len(res_train))

model_svm_reg = svmutil.svm_train(res_train, train,'-s 4 -v 10 -t 2 -c 2')

In [0]:
model_svm = svmutil.svm_train(res_train, train,'-s 4 -t 2 -c 2')
y_reg = svmutil.svm_predict(res_test+res_train, test+train, model_svm)
svmutil.svm_save_model("model_svm_reg.bin",model_svm)

from sklearn.metrics import r2_score, mean_absolute_error

In [0]:
r2 = r2_score(res_test+res_train,y_reg[0])
r2

In [0]:
mae = mean_absolute_error(res_test+res_train, y_reg[0])
mae

In [0]:
from scipy.stats import kendalltau
kt = kendalltau(res_test+res_train, y_reg[0])
kt

In [0]:
y_reg2 = svmutil.svm_predict(res_train, train, model_svm)

In [0]:
r2 = r2_score(res_train,y_reg2[0])
r2

In [0]:
mae = mean_absolute_error(res_train, y_reg2[0])
mae

In [0]:
kt = kendalltau(res_train, y_reg2[0])
kt

In [0]:
y_reg3 = svmutil.svm_predict(res_test, test, model_svm)

In [0]:
r2 = r2_score(res_test,y_reg3[0])
r2

In [0]:
mae = mean_absolute_error(res_test, y_reg3[0])
mae

In [0]:
from scipy.stats import kendalltau
kt = kendalltau(res_test, y_reg3[0])
kt

**Classification : Relevant News Category Features**

In [0]:
data = pd.read_excel("NewsHeadlines_01012019_30092019.xlsx")
ls = [1 if i < 4761 else 0 for i in range(47611)]
# data.set_index("Like Count", inplace=True)
data.sort_values("Like Count", inplace=True,ascending=False)
# print(data.head())
data['class'] = ls
index = data["Like Count"].tolist()
data["Category"].replace({"Tech": "Technology","-":"Uncategorized", "Sports":"Sport"}, inplace=True)
data.head()

In [0]:
model_summary = FastText.load("model_summary.bin")

In [0]:
category = set(data["Category"].values.tolist())
headlines= []
for sent_str in data["Headline"]:
#     sent_str = re.sub(r"'", "", str(sent_str).lower())
    tokens = re.sub(r"[^a-z0-9]+", " ", str(sent_str).lower()).split()
    if(tokens == []):
        tokens = "nan"
    headlines.append(tokens)
    
cat_vec = {}
for c in category:
    cat_vec[c] = model_summary[c]    
    
head_vec = []
n = len(headlines)
for h in range(n):
    sum1 = np.array([0.0 for i in range(100)])
    for tok in headlines[h]:
        sum1+=model_summary[tok]
#     print(sum1)
    sum1 = sum1/len(headlines[h])
#     print(sum1)
    sum1 = np.concatenate((sum1,cat_vec[data["Category"][h]]))
    head_vec.append(sum1)

In [0]:
train = head_vec[:4261]+head_vec[4761:4761*2]
res_train = data["class"][:4261].tolist()+data["class"][4761:4761*2].tolist()
test = head_vec[4261:4761]+head_vec[int(0.8*47611):]
res_test = data["class"][4261:4761].tolist()+data["class"][int(0.8*47611):].tolist()
print(len(train), len(res_train))

In [0]:
model_svm_5050 = svmutil.svm_train(res_train, train,'-t 2 -c 2')

In [0]:
y_50 = svmutil.svm_predict(res_test, test, model_svm_5050)

In [0]:
y_1 = svmutil.svm_predict(res_test[:500], test[:500], model_svm_5050)

In [0]:
from sklearn.metrics import confusion_matrix, auc, roc_curve

In [0]:
confusion_matrix(res_test, y_50[0], labels=[0,1])

In [0]:
tn, fp, fn, tp = confusion_matrix(res_test, y_50[0], labels=[0,1]).ravel()

In [0]:
recall = tp/(tp+fn)
recall

In [0]:
precision = tp/(tp+fp)
precision

In [0]:
f1 = precision*recall*2/(precision+recall)
f1

In [0]:
fpr, tpr, thresholds = roc_curve(res_test, y_50[0])
AUC = auc(fpr, tpr)
AUC

In [0]:
svmutil.svm_save_model("model_svm.bin",model_svm_5050)

In [0]:
svmutil.svm_train(res_train, train,'-v 10 -t 2 -c 2')

**Regression: Named Entity Features**

In [0]:
import pandas as pd
from gensim.models import FastText
import re
import numpy as np
from libsvm import svmutil
import libsvm
from sklearn.metrics import r2_score, mean_absolute_error
from scipy.stats import kendalltau

In [0]:
model_summary = FastText.load("models-4.1(a)/model_summary.bin")
data = pd.read_csv("data_ner.csv", sep=";")
headlines = data["ner_tokens"].astype(str).tolist()

In [0]:
head_vec = []
n = len(headlines)
for h in range(n):
    sum1 = np.array([0.0 for i in range(100)])
    for tok in headlines[h]:
        sum1+=model_summary[tok[0]]
#     print(sum1)
    sum1 = sum1/len(headlines[h])
#     print(sum1)
    head_vec.append(sum1)

In [0]:
feat_len = int(0.8*len(head_vec))
train = head_vec[:feat_len]
res_train = list(np.log2(np.array(data["Like Count"][:feat_len].tolist())+1))
test = head_vec[feat_len:]
res_test = list(np.log2(np.array(data["Like Count"][feat_len:].tolist())+1))
print(len(train), len(res_train))

In [0]:
model_svm = svmutil.svm_train(res_train, train,'-s 4 -t 2 -c 2')

In [0]:
y_reg = svmutil.svm_predict(res_test, test, model_svm)

In [0]:
r2 = r2_score(res_test,y_reg[0])
r2

In [0]:
mae = mean_absolute_error(res_test, y_reg[0])
mae

In [0]:
kt = kendalltau(res_test, y_reg[0])
kt

In [0]:
y_full = svmutil.svm_predict(res_test+res_train, test+train, model_svm)

In [0]:
r2 = r2_score(res_test+res_train,y_full[0])
r2

In [0]:
mae = mean_absolute_error(res_test+res_train, y_full[0])
mae

In [0]:
kt = kendalltau(res_test+res_train, y_full[0])
kt

In [0]:
y_tr = svmutil.svm_predict(res_train, train, model_svm)

In [0]:
r2 = r2_score(res_train,y_tr[0])
r2

In [0]:
mae = mean_absolute_error(res_train, y_tr[0])
mae

In [0]:
kt = kendalltau(res_train, y_tr[0])
kt

In [0]:
model_svm_reg = svmutil.svm_train(res_train+res_test, train+test,'-s 4 -v 10 -t 2 -c 2')

In [0]:
svmutil.svm_save_model("model_b_reg.bin",model_svm)

**Classification: Named Entity Feature**

In [0]:
from nltk.tag import StanfordNERTagger

In [0]:
model_summary = FastText.load("models-4.1(a)/model_summary.bin")

In [0]:
st = StanfordNERTagger('stanford-ner-2018-10-16/classifiers/english.all.3class.distsim.crf.ser.gz', 'stanford-ner-2018-10-16/stanford-ner.jar') # doctest: +SKIP

In [0]:
data = pd.read_excel("NewsHeadlines_01012019_30092019.xlsx")
ls = [1 if i < 4761 else 0 for i in range(47611)]
# data.set_index("Like Count", inplace=True)
data.sort_values("Like Count", inplace=True,ascending=False)
data['class'] = ls
data.reset_index(inplace=True,drop=True)
data = data.truncate(before=0,after=4761*2)
data = data.sample(frac=1)

In [0]:
headlines= []
i= 0
for sent_str in data["Headline"]:
    print(i)
    i+=1
    tokens = re.sub(r"[^A-Za-z0-9]+", " ", str(sent_str)).split()
    if(tokens == []):
        tokens = "nan"
    tokens = st.tag(tokens)
#     print(tokens)
    headlines.append([])
    for t in tokens:
        if(t[1]!='O'):
            headlines[-1].append(t)
    if(len(headlines[-1]) == 0):
        headlines[-1] = "nan"
    


In [0]:
head_vec = []
n = len(headlines)
for h in range(n):
    sum1 = np.array([0.0 for i in range(100)])
    for tok in headlines[h]:
        sum1+=model_summary[tok[0]]
    sum1 = sum1/len(headlines[h])
    head_vec.append(sum1)


In [0]:
data["ner_tokens"] = headlines

In [0]:
feat_len = int(0.8*len(head_vec))
train = head_vec[:feat_len]
res_train = data["class"][:feat_len].tolist()
test = head_vec[feat_len::]
res_test = data["class"][feat_len:].tolist()
print(len(train), len(res_train))

In [0]:
model_svm = svmutil.svm_train(res_train, train,'-t 2 -c 2')

In [0]:

y = svmutil.svm_predict(res_test, test, model_svm)

In [0]:
confusion_matrix(res_test, y[0], labels=[0,1])

In [0]:
tn, fp, fn, tp = confusion_matrix(res_test, y[0], labels=[0,1]).ravel()

In [0]:
recall = tp/(tp+fn)
recall

In [0]:
precision = tp/(tp+fp)
precision

In [0]:
f1 = precision*recall*2/(precision+recall)
f1

In [0]:
fpr, tpr, thresholds = roc_curve(res_test, y[0])
AUC = auc(fpr, tpr)
AUC

In [0]:
svmutil.svm_save_model("model_b_classification.bin",model_svm)

In [0]:
svmutil.svm_train(res_train, train,'-v 10 -t 2 -c 2')

**Regression: Richness Post Type Features**

In [0]:
model_summary = FastText.load("models-4.1(a)/model_summary.bin")

data = pd.read_excel("NewsHeadlines_01012019_30092019.xlsx")
data.columns =[column.replace(" ", "_") for column in data.columns]
data.query(' Post_Type == "photo" or Post_Type == "video" or Post_Type == "link" ', inplace=True)
data.reset_index(inplace=True, drop=True)

post_types = set(data["Post_Type"].tolist())
headlines= []
for sent_str in data["Headline"]:
#     sent_str = re.sub(r"'", "", str(sent_str).lower())
    tokens = re.sub(r"[^a-z0-9]+", " ", str(sent_str).lower()).split()
    if(tokens == []):
        tokens = "nan"
    headlines.append(tokens)
    
cat_vec = {}
for c in post_types:
    cat_vec[c] = model_summary[c]    
    
head_vec = []
n = len(headlines)
for h in range(n):
    sum1 = np.array([0.0 for i in range(100)])
    for tok in headlines[h]:
        sum1+=model_summary[tok]
#     print(sum1)
    sum1 = sum1/len(headlines[h])
#     print(sum1)
    sum1 = np.concatenate((sum1,cat_vec[data["Post_Type"][h]]))
    head_vec.append(sum1)

feat_len = int(0.8*len(head_vec))
train = head_vec[:feat_len]
res_train = list(np.log2(np.array(data["Like_Count"][:feat_len].tolist())+1))
test = head_vec[feat_len::]
res_test = list(np.log2(np.array(data["Like_Count"][feat_len:].tolist())+1))
print(len(train), len(res_train))

svmutil.svm_save_model("model_c_reg.bin",model_svm)

In [0]:
y_reg = svmutil.svm_predict(res_test, test, model_svm)

In [0]:
r2 = r2_score(res_test,y_reg[0])
r2

In [0]:
mae = mean_absolute_error(res_test, y_reg[0])
mae

In [0]:
kt = kendalltau(res_test, y_reg[0])
kt

In [0]:
y_full = svmutil.svm_predict(res_test+res_train, test+train, model_svm)

In [0]:
r2 = r2_score(res_test+res_train,y_full[0])
r2

In [0]:
mae = mean_absolute_error(res_test+res_train, y_full[0])
mae

In [0]:
kt = kendalltau(res_test+res_train, y_full[0])
kt

In [0]:
y_tr = svmutil.svm_predict(res_train, train, model_svm)

In [0]:
r2 = r2_score(res_train,y_tr[0])
r2

In [0]:
mae = mean_absolute_error(res_train, y_tr[0])
mae

In [0]:
kt = kendalltau(res_train, y_tr[0])
kt

**Classification: Richnes Post Type Feature**

In [0]:
model_summary = FastText.load("models-4.1(a)/model_summary.bin")

In [0]:
data = pd.read_excel("NewsHeadlines_01012019_30092019.xlsx")
data.columns =[column.replace(" ", "_") for column in data.columns] 
ls = [1 if i < 4761 else 0 for i in range(47611)]
# data.set_index("Like Count", inplace=True)
data.sort_values("Like_Count", inplace=True,ascending=False)
data['class'] = ls
data.query(' Post_Type == "photo" or Post_Type == "video" or Post_Type == "link" ', inplace=True)
data.reset_index(inplace=True, drop=True)
data = data.truncate(before=0,after=4761*2)
data = data.sample(frac=1)

post_types = set(data["Post_Type"].tolist())
headlines= []
for sent_str in data["Headline"]:
#     sent_str = re.sub(r"'", "", str(sent_str).lower())
    tokens = re.sub(r"[^a-z0-9]+", " ", str(sent_str).lower()).split()
    if(tokens == []):
        tokens = "nan"
    headlines.append(tokens)
    
cat_vec = {}
for c in post_types:
    cat_vec[c] = model_summary[c]    
    
head_vec = []
n = len(headlines)
for h in range(n):
    sum1 = np.array([0.0 for i in range(100)])
    for tok in headlines[h]:
        sum1+=model_summary[tok]
#     print(sum1)
    sum1 = sum1/len(headlines[h])
#     print(sum1)
    sum1 = np.concatenate((sum1,cat_vec[data["Post_Type"][h]]))
    head_vec.append(sum1)

feat_len = int(0.8*len(head_vec))
train = head_vec[:feat_len]
res_train = data["class"][:feat_len].tolist()
test = head_vec[feat_len::]
res_test = data["class"][feat_len:].tolist()
print(len(train), len(res_train))    

In [0]:
model_svm_50100 = svmutil.svm_train(res_train, train,'-t 2 -c 2')

In [0]:
y = svmutil.svm_predict(res_test, test, model_svm_50100)

In [0]:
confusion_matrix(res_test, y[0], labels=[0,1])

In [0]:
tn, fp, fn, tp = confusion_matrix(res_test, y[0], labels=[0,1]).ravel()

In [0]:
recall = tp/(tp+fn)
recall

In [0]:
precision = tp/(tp+fp)
precision

In [0]:
f1 = precision*recall*2/(precision+recall)
f1

In [0]:
fpr, tpr, thresholds = roc_curve(res_test, y[0])
AUC = auc(fpr, tpr)
AUC

In [0]:
svmutil.svm_save_model("model_b_svm.bin",model_svm_50100)

In [0]:
svmutil.svm_train(res_train, train,'-v 10 -t 2 -c 2')

**Classification: Linguistics Style Feature**

In [0]:
model_summary = FastText.load("models-4.1(a)/model_summary.bin")
st = StanfordPOSTagger('stanford-postagger-2018-10-16/models/english-bidirectional-distsim.tagger', 'stanford-postagger-2018-10-16/stanford-postagger.jar') # doctest: +SKIP

In [0]:
data = pd.read_excel("NewsHeadlines_01012019_30092019.xlsx")
ls = [1 if i < 4761 else 0 for i in range(47611)]
data.sort_values("Like Count", inplace=True,ascending=False)
data['class'] = ls
data.query(' Headline != "-" ', inplace=True)
data.reset_index(inplace=True, drop=True)
data = data.truncate(before=0,after=4761*2)
data = data.sample(frac=1)

In [0]:
def get_vector(h):
    
    headlines= []
    i= 0
    for sent_str in h:

        print(i)

        i+=1

        sent_str = str(sent_str)

        headlines.append([0,0,0,0,0,0])

        headlines[-1][0] = len(sent_str)

        tokens = str(sent_str).split()

        headlines[-1][1] = len(tokens)

        if(tokens == []):
            print(sent_str, " error sentence ")

        tokens = st.tag(tokens)
#         print(tokens)

        for t in tokens:

            if(t[1] == 'JJR'):
                headlines[-1][4] += 1

            if(t[1] == 'JJS'):
                headlines[-1][5] += 1

            if(len(t[0]) > 6):
                headlines[-1][2] += 1

            if(t[0] == t[0].upper()):
                headlines[-1][3] += 1
    
    return headlines

In [0]:
dic = {'-' : 0, 'nan' : 0}
for sent in data['Headline']:
    if(str(sent) in dic):
        dic[str(sent)]+=1
#         print(sent)

In [0]:
head_vec = get_vector(data['Headline'])

In [0]:
head_vec = np.array(head_vec)
head_vec.shape

In [0]:
np.save("part_d_features", head_vec)

In [0]:
data.to_csv("data_pos.csv")

In [0]:
feat_len = int(0.8*len(head_vec))
train = head_vec[:feat_len]
res_train = data["class"][:feat_len].tolist()
test = head_vec[feat_len::]
res_test = data["class"][feat_len:].tolist()
print(len(train), len(res_train))

In [0]:
model_svm = svmutil.svm_train(res_train, train,'-t 2 -c 2')

In [0]:
y = svmutil.svm_predict(res_test, test, model_svm)

In [0]:
confusion_matrix(res_test, y[0], labels=[0,1])

In [0]:
tn, fp, fn, tp = confusion_matrix(res_test, y[0], labels=[0,1]).ravel()
recall = tp/(tp+fn)
recall

In [0]:
precision = tp/(tp+fp)
precision

In [0]:
f1 = precision*recall*2/(precision+recall)
f1

In [0]:
fpr, tpr, thresholds = roc_curve(res_test, y[0])
AUC = auc(fpr, tpr)
AUC

In [0]:
svmutil.svm_save_model("model_d_classification.bin",model_svm)

In [0]:
svmutil.svm_train(res_train+res_test, train+test,'-v 10 -t 2 -c 2')

**Regression: Linguistics Style Feature**

In [0]:
model_summary = FastText.load("models-4.1(a)/model_summary.bin")
data = pd.read_csv("data_pos.csv")
head_vec = np.load("part_d_features.npy")

In [0]:
feat_len = int(0.8*len(head_vec))
train = head_vec[:feat_len]
res_train = list(np.log2(np.array(data["Like Count"][:feat_len].tolist())+1))
test = head_vec[feat_len:]
res_test = list(np.log2(np.array(data["Like Count"][feat_len:].tolist())+1))
print(len(train), len(res_train))

In [0]:
model_svm = svmutil.svm_train(res_train, train,'-s 4 -t 2 -c 2')

In [0]:
y_reg = svmutil.svm_predict(res_test, test, model_svm)

In [0]:
r2 = r2_score(res_test,y_reg[0])
r2

In [0]:
mae = mean_absolute_error(res_test, y_reg[0])
mae

In [0]:
kt = kendalltau(res_test, y_reg[0])
kt

In [0]:
y_full = svmutil.svm_predict(res_test+res_train, list(test)+list(train), model_svm)

In [0]:
r2 = r2_score(res_test+res_train,y_full[0])
r2

In [0]:
mae = mean_absolute_error(res_test+res_train, y_full[0])
mae

In [0]:
kt = kendalltau(res_test+res_train, y_full[0])
kt

In [0]:
y_tr = svmutil.svm_predict(res_train, train, model_svm)

In [0]:
r2 = r2_score(res_train,y_tr[0])
r2

In [0]:
mae = mean_absolute_error(res_train, y_tr[0])
mae

In [0]:
kt = kendalltau(res_train, y_tr[0])
kt

In [0]:
model_svm_reg = svmutil.svm_train(res_train+res_test, list(train)+list(test),'-s 4 -v 10 -t 2 -c 2')

In [0]:
svmutil.svm_save_model("model_d_reg.bin",model_svm)